In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv('dataset/Train-1542865627584.csv')
train_be = pd.read_csv('dataset/Train_Beneficiarydata-1542865627584.csv')
train_ip = pd.read_csv('dataset/Train_Inpatientdata-1542865627584.csv')
train_op = pd.read_csv('dataset/Train_Outpatientdata-1542865627584.csv')

In [3]:
data = pd.concat([train_op, train_ip])
data = pd.merge(data, train_be, on='BeneID')
data = pd.merge(data, train, on='Provider')

data.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode,AdmissionDt,DischargeDt,DiagnosisGroupCode,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,V5866,V1272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,56409,NaN,NaN,NaN,1936-09-01,NaN,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50,Yes
1,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1922-07-01,NaN,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,Yes
2,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,30000,72887,4280,7197,V4577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,71947,NaN,NaN,NaN,1922-07-01,NaN,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,Yes
3,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1922-07-01,NaN,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,Yes
4,BENE11004,CLM224741,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,4280,2449,V854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1922-07-01,NaN,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,Yes


In [4]:
df = data.copy()

data.drop(['ClaimID','ClmProcedureCode_5', 'ClmProcedureCode_6'], axis=1, inplace=True)

# Feature Engineering
In the following cells, we will engineer the variables of the House Price Dataset so that we tackle:

1. Missing values
2. Temporal variables
3. Non-Gaussian distributed variables
4. Categorical variables: remove rare labels
5. Categorical variables: convert strings to numbers
6. Put the variables in a similar scale

In [10]:
date_vars = [var for var in data.columns if 'Dt' in var] + ['DOB','DOD']

cat_vars = [var for var in data.columns if data[var].dtype=='O' and var not in date_vars+['PotentialFraud']] \
            + [var for var in data.columns if data[var].dtype!='O' and 'Code' in var] \
            + ['Gender','Race','State','County','NoOfMonths_PartACov','NoOfMonths_PartBCov']

num_vars = [var for var in data.columns if var not in date_vars+cat_vars+['PotentialFraud']]

## Missing values
### Categorical variables
We will replace missing values with the string "missing" in those variables with a lot of missing data.

Alternatively, we will replace missing data with the most frequent category in those variables that contain fewer observations without values.

This is common practice.

In [11]:
data[cat_vars] = data[cat_vars].astype('O')

In [12]:
cats_with_na = [var for var in cat_vars if data[var].isnull().mean()>0]

data[cats_with_na].isnull().mean().sort_values(ascending=False)

ClmProcedureCode_4       0.999789
ClmProcedureCode_3       0.998264
ClmDiagnosisCode_10      0.991025
ClmProcedureCode_2       0.990165
ClmProcedureCode_1       0.958242
DiagnosisGroupCode       0.927493
ClmDiagnosisCode_9       0.925091
ClmDiagnosisCode_8       0.904258
ClmDiagnosisCode_7       0.881448
ClmDiagnosisCode_6       0.848817
ClmDiagnosisCode_5       0.799495
OperatingPhysician       0.794975
ClmAdmitDiagnosisCode    0.738631
ClmDiagnosisCode_4       0.705244
OtherPhysician           0.642185
ClmDiagnosisCode_3       0.564582
ClmDiagnosisCode_2       0.350416
ClmDiagnosisCode_1       0.018726
AttendingPhysician       0.002701
dtype: float64

In [13]:
# variables to impute with the string missing
with_string_missing = [
    var for var in cats_with_na if data[var].isnull().mean() > 0.1]

# variables to impute with the most frequent category
with_frequent_category = [
    var for var in cats_with_na if data[var].isnull().mean() < 0.1]

In [15]:
with_string_missing

['OperatingPhysician',
 'OtherPhysician',
 'ClmDiagnosisCode_2',
 'ClmDiagnosisCode_3',
 'ClmDiagnosisCode_4',
 'ClmDiagnosisCode_5',
 'ClmDiagnosisCode_6',
 'ClmDiagnosisCode_7',
 'ClmDiagnosisCode_8',
 'ClmDiagnosisCode_9',
 'ClmDiagnosisCode_10',
 'ClmAdmitDiagnosisCode',
 'DiagnosisGroupCode',
 'ClmProcedureCode_1',
 'ClmProcedureCode_2',
 'ClmProcedureCode_3',
 'ClmProcedureCode_4']

In [16]:
# replace missing values with new label: "Missing"

data[with_string_missing] = data[with_string_missing].fillna('Missing')

In [17]:
for var in with_frequent_category:
    
    # there can be more than 1 mode in a variable
    # we take the first one with [0]    
    mode = data[var].mode()[0]
    
    print(var, mode)
    
    data[var].fillna(mode, inplace=True)

AttendingPhysician PHY330576
ClmDiagnosisCode_1 4019


In [18]:
# check that test set does not contain null values in the engineered variables

[var for var in cats_with_na if data[var].isnull().sum() > 0]

[]

### Numerical variables
To engineer missing values in numerical variables, we will:

add a binary missing indicator variable
and then replace the missing values in the original variable with the mean

In [20]:
nums_with_na = [var for var in num_vars if data[var].isnull().sum()>0]
nums_with_na

['DeductibleAmtPaid']

In [21]:
# replace missing values as we described above

for var in nums_with_na:

    # calculate the mean using the train set
    mean_val = data[var].mean()
    
    print(var, mean_val)

    # add binary missing indicator (in train and test)
    data[var + '_na'] = np.where(data[var].isnull(), 1, 0)

    # replace missing values by the mean
    # (in train and test)
    data[var].fillna(mean_val, inplace=True)

# check that we have no more missing values in the engineered variables
data[nums_with_na].isnull().sum()

DeductibleAmtPaid 78.42108549609554


DeductibleAmtPaid    0
dtype: int64

## Temporal variables
### Capture elapsed time
We learned in the previous notebook, that there are 4 variables that refer to the years in which the house or the garage were built or remodeled.

We will capture the time elapsed between those variables and the year in which the house was sold:

Sebelum kita melakukan imputasi kita

In [22]:
for var in date_vars:
    data[var] = pd.to_datetime(data[var])

In [23]:
data[date_vars].head()

,ClaimStartDt,ClaimEndDt,AdmissionDt,DischargeDt,DOB,DOD
0,2009-10-11,2009-10-11,NaT,NaT,1936-09-01,NaT
1,2009-01-06,2009-01-06,NaT,NaT,1922-07-01,NaT
2,2009-01-22,2009-01-22,NaT,NaT,1922-07-01,NaT
3,2009-02-03,2009-02-03,NaT,NaT,1922-07-01,NaT
4,2009-03-03,2009-03-03,NaT,NaT,1922-07-01,NaT


In [29]:
data['ClaimYr'] = data['ClaimStartDt'].dt.year
data['ClaimDur'] = (data['ClaimEndDt']-data['ClaimStartDt']).dt.days
data['StayDur'] = (data['DischargeDt'] - data['AdmissionDt']).dt.days
data['AgeAtClaim'] = (data['ClaimStartDt'] - data['DOB']).dt.days
data['IsAlive'] = [1 if pd.isna(i) else 0 for i in data['DOD']]

In [30]:
data.drop(date_vars, axis=1, inplace=True)

In [31]:
data[['ClaimYr','ClaimDur','StayDur','AgeAtClaim','IsAlive']].head()

,ClaimYr,ClaimDur,StayDur,AgeAtClaim,IsAlive
0,2009,0,NaN,26703,1
1,2009,0,NaN,31601,1
2,2009,0,NaN,31617,1
3,2009,0,NaN,31629,1
4,2009,0,NaN,31657,1


In [32]:
data[['ClaimYr','ClaimDur','StayDur','AgeAtClaim','IsAlive']].isnull().sum()

ClaimYr            0
ClaimDur           0
StayDur       517737
AgeAtClaim         0
IsAlive            0
dtype: int64

In [39]:
# add binary missing indicator (in train and test)
data['StayDur' + '_na'] = np.where(data[var].isnull(), 1, 0)

# replace missing values by 0
data['StayDur'].fillna(0, inplace=True)

In [40]:
[var for var in data.columns if data[var].isnull().mean()>0]

[]

In [41]:
data.head()

,BeneID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,DeductibleAmtPaid,ClmAdmitDiagnosisCode,DiagnosisGroupCode,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud,DeductibleAmtPaid_na,ClaimYr,ClaimDur,StayDur,AgeAtClaim,IsAlive,StayDur_na
0,BENE11002,PRV56011,30,PHY326117,Missing,Missing,78943,V5866,V1272,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,0.0,56409,Missing,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50,Yes,0,2009,0,0.0,26703,1,0
1,BENE11004,PRV56011,40,PHY334319,Missing,Missing,71988,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,0.0,Missing,Missing,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,Yes,0,2009,0,0.0,31601,1,0
2,BENE11004,PRV56011,200,PHY403831,Missing,Missing,82382,30000,72887,4280,7197,V4577,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,0.0,71947,Missing,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,Yes,0,2009,0,0.0,31617,1,0
3,BENE11004,PRV56011,20,PHY339887,Missing,Missing,20381,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,0.0,Missing,Missing,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,Yes,0,2009,0,0.0,31629,1,0
4,BENE11004,PRV56011,40,PHY345721,Missing,Missing,V6546,4280,2449,V854,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,0.0,Missing,Missing,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,Yes,0,2009,0,0.0,31657,1,0


In [43]:
data[cat_vars].nunique()

BeneID                   138556
Provider                   5410
AttendingPhysician        82063
OperatingPhysician        35316
OtherPhysician            46458
ClmDiagnosisCode_1        10450
ClmDiagnosisCode_2         5301
ClmDiagnosisCode_3         4757
ClmDiagnosisCode_4         4360
ClmDiagnosisCode_5         3971
ClmDiagnosisCode_6         3608
ClmDiagnosisCode_7         3389
ClmDiagnosisCode_8         3071
ClmDiagnosisCode_9         2775
ClmDiagnosisCode_10        1159
ClmAdmitDiagnosisCode      4099
DiagnosisGroupCode          737
RenalDiseaseIndicator         2
ClmProcedureCode_1         1118
ClmProcedureCode_2          301
ClmProcedureCode_3          155
ClmProcedureCode_4           49
Gender                        2
Race                          4
State                        52
County                      314
NoOfMonths_PartACov          13
NoOfMonths_PartBCov          13
dtype: int64

In [44]:
len(data)

558211

In [49]:
data['BeneID'].value_counts()

BeneID
BENE42721     29
BENE59303     29
BENE118316    29
BENE36330     28
BENE80977     28
              ..
BENE78844      1
BENE81386      1
BENE84188      1
BENE85299      1
BENE105577     1
Name: count, Length: 138556, dtype: int64

In [59]:
len(data['ClmDiagnosisCode_1'].value_counts()/len(data))

10450